# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [57]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

In [58]:
bb.get_version_string()

'4.4.0'

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [59]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut4_simple'
rtl_module_name       = 'MnistLut4Simple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [60]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [61]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
#           bb.DifferentiableLut([4*4*64], connection='random', N=4),
            bb.DifferentiableLut([4*64],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

            bb.DifferentiableLut([4*64],   connection='random', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

            bb.DifferentiableLut([4*64],   connection='random', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

#           bb.DifferentiableLut([2*4*4*10], connection='random', N=4),
            bb.DifferentiableLut([4*4*10],  connection='random', N=4),
            bb.DifferentiableLut([4*10],   connection='serial', N=4),
#           bb.DifferentiableLut([10],     connection='serial', N=4),
            bb.AverageLut([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [62]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch[0] : loss=1.666878 accuracy=0.787300


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[1] : loss=1.661615 accuracy=0.802500


  0%|          | 0/938 [00:00<?, ?it/s]

...

epoch[5] : loss=1.700348 accuracy=0.777700


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[6] : loss=1.720199 accuracy=0.718400


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[7] : loss=1.646675 accuracy=0.844600


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [63]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="", ff=False)

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut4_simple/MnistLut4Simple.v'

In [64]:
output_velilog_file

'./data/MnistDifferentiableLut4Simple/MnistLut4Simple.v'

In [65]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [66]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [67]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [ 96, 287, 745, 140],
       [ 39,  75, 762,  24],
       [730, 179,  33, 337]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [68]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 16)


array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False,  True, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ...,  True,  True,  True]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [69]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 16)


array([[0.54629767, 0.63571876, 0.4969459 , ..., 0.37840033, 0.49704102,
        0.49817133],
       [0.5361756 , 0.6309467 , 0.5341011 , ..., 0.44865036, 0.43429276,
        0.4580546 ],
       [0.58685493, 0.47376254, 0.3866231 , ..., 0.45542198, 0.46202305,
        0.42041367],
       ...,
       [0.50848675, 0.41154054, 0.56788254, ..., 0.47694454, 0.562672  ,
        0.4481822 ],
       [0.5978139 , 0.52566504, 0.49524385, ..., 0.46574724, 0.42562854,
        0.42392486],
       [0.4355895 , 0.4515173 , 0.5214482 , ..., 0.5049831 , 0.5609362 ,
        0.55069613]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [70]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 16)


array([[ 1.4218781e-04,  4.7396054e-05, -4.5536290e-05, ...,
         8.4025996e-06, -5.7424666e-05, -1.9141551e-05],
       [-3.7252903e-09, -2.3283064e-10, -2.3283064e-10, ...,
        -4.6566129e-10, -4.6566129e-10, -8.7311491e-11],
       [ 3.0960427e-03,  1.0334138e-03,  1.3408652e-03, ...,
        -3.9462995e-04, -2.9648328e-04, -9.4625932e-05],
       ...,
       [-5.4481616e-03,  3.7673465e-04,  3.2253526e-03, ...,
         4.2970496e-05,  3.6837198e-04,  2.0400502e-04],
       [-9.3132257e-10,  6.9849193e-10,  6.9849193e-10, ...,
        -2.3283064e-10, -2.3283064e-10, -2.9103830e-11],
       [ 9.9895988e-07,  3.3236574e-07, -9.5542055e-07, ...,
         3.6987331e-08, -1.0614167e-07, -3.5361154e-08]], dtype=float32)